### 1、商户&&店铺的关系
#### 统计一下每个merchant_id有多少store

#### 新特征 store_count

In [2]:
import pandas as pd

# 加载数据
data = pd.read_csv('流水数据/combined_data_unique.txt')

# 删除 merchant_id 和 store_id 都相同的重复行
data.drop_duplicates(subset=['merchant_id', 'store_id'], keep='first', inplace=True)

# 计算每个 merchant_id 对应的不同 store_id 的数量
store_counts = data.groupby('merchant_id')['store_id'].nunique().reset_index(name='store_count')

# 保存结果
store_counts.to_csv('商户-店铺/merchant_count.txt', index=False)

# 打印部分数据以确认
print(store_counts.head())


    merchant_id  store_count
0  164831531007            1
1  217979763279            1
2  454350470085            1
3  567722732408            1
4  762310083866            1


In [2]:
# 读取文件
with open('商户-店铺/merchant_count.txt', 'r') as file:
    lines = file.readlines()

# 初始化计数器
count = 0

# 遍历每一行数据，跳过第一行
for line in lines[1:]:
    merchant_id, store_count = line.strip().split(',')
    if int(store_count) != 1:
        count += 1

# 打印结果
print(count)


228


In [3]:
import pandas as pd

# 读取数据文件
file_path = '流水数据/combined_data_unique.txt'
data = pd.read_csv(file_path, usecols=['store_id', 'merchant_id'])

# 去除重复的 (merchant_id, store_id) 组合
data = data.drop_duplicates(subset=['merchant_id', 'store_id'])

# 对每个 merchant_id 计算不同的 store_id 数量
merchant_store_count = data.groupby('merchant_id')['store_id'].nunique().reset_index()
merchant_store_count.rename(columns={'store_id': 'store_count'}, inplace=True)

# 将计数结果合并回原始数据
result = pd.merge(data, merchant_store_count, on='merchant_id')

# 保存新的数据文件
output_file = '商户-店铺/merchant_store_count.txt'
result.to_csv(output_file, index=False)

# 打印整体数据量和前三行数据
print(f"整体数据量: {result.shape[0]}")
print("前三行数据：")
print(result.head(3))


整体数据量: 5933
前三行数据：
          store_id      merchant_id  store_count
0  292289830769797  516101438346939            1
1  753408387511040  965952426216589            1
2  487886544602057  865666381695711            1


In [4]:
#check
import pandas as pd

# 加载商户-店铺数据
store_data = pd.read_csv('商户-店铺/merchant_store_count.txt')

# 按 merchant_id 分组，并计算每个组中 store_count 的唯一值数量
grouped = store_data.groupby('merchant_id')['store_count'].nunique()

# 找出 store_count 唯一值数量大于1的 merchant_id
inconsistent_merchants = grouped[grouped > 1]

# 打印结果
if len(inconsistent_merchants) > 0:
    print("存在不一致的merchant_id及其对应的store_count数量:")
    print(inconsistent_merchants)
else:
    print("所有相同的merchant_id对应的store_count是一致的。")


所有相同的merchant_id对应的store_count是一致的。


### 2、查看是否有商户没有store_count的信息
#### 发现没有历史还款记录、即没有历史还款也没有交易流水的商户，一定没有store_count的信息

In [19]:
import pandas as pd

# 文件路径
test_files = [
    '测试1/test_both_with_merchant_info.txt',
    '测试1/test_combined_with_merchant_info.txt',
    '测试1/test_history_with_merchant_info.txt',
    '测试1/test_neither_with_merchant_info.txt'
]
merchant_count_file = '商户-店铺/merchant_count.txt'

# 读取商户ID数据
merchant_count_df = pd.read_csv(merchant_count_file)
merchant_ids_set = set(merchant_count_df['merchant_id'])

# 分别处理每个测试文件
for test_file in test_files:
    test_df = pd.read_csv(test_file)
    # 计算不能在merchant_count.txt中找到的merchant_id数量
    missing_count = test_df[~test_df['merchant_id'].isin(merchant_ids_set)].shape[0]
    print(f"{test_file} 中不能找到的merchant_id数量为: {missing_count}")


测试1/test_both_with_merchant_info.txt 中不能找到的merchant_id数量为: 0
测试1/test_combined_with_merchant_info.txt 中不能找到的merchant_id数量为: 0
测试1/test_history_with_merchant_info.txt 中不能找到的merchant_id数量为: 6
测试1/test_neither_with_merchant_info.txt 中不能找到的merchant_id数量为: 11


In [20]:
import pandas as pd

# 文件路径
test_files = [
    '训练1/train_both_with_merchant_info.txt',
    '训练1/train_combined_with_merchant_info.txt',
    '训练1/train_history_with_merchant_info.txt',
    '训练1/train_neither_with_merchant_info.txt'
]
merchant_count_file = '商户-店铺/merchant_count.txt'

# 读取商户ID数据
merchant_count_df = pd.read_csv(merchant_count_file)
merchant_ids_set = set(merchant_count_df['merchant_id'])

# 分别处理每个测试文件
for test_file in test_files:
    test_df = pd.read_csv(test_file)
    # 计算不能在merchant_count.txt中找到的merchant_id数量
    missing_count = test_df[~test_df['merchant_id'].isin(merchant_ids_set)].shape[0]
    print(f"{test_file} 中不能找到的merchant_id数量为: {missing_count}")


训练1/train_both_with_merchant_info.txt 中不能找到的merchant_id数量为: 0
训练1/train_combined_with_merchant_info.txt 中不能找到的merchant_id数量为: 0
训练1/train_history_with_merchant_info.txt 中不能找到的merchant_id数量为: 22
训练1/train_neither_with_merchant_info.txt 中不能找到的merchant_id数量为: 36


### 3、填充store_count特征

#####      测试数据

In [21]:
import pandas as pd

# 加载商户-店铺数据并确保merchant_id唯一
store_data = pd.read_csv('商户-店铺/merchant_count.txt')
store_data = store_data.drop_duplicates(subset=['merchant_id'], keep='first')

# 测试文件路径
test_files = [
    '测试1/test_both_with_merchant_info.txt',
    '测试1/test_combined_with_merchant_info.txt'
]

# 处理每个测试文件
for test_file in test_files:
    # 读取测试数据
    test_data = pd.read_csv(test_file)
    
    # 确保测试数据中的merchant_id在商户-店铺数据中能找到
    test_data = test_data[test_data['merchant_id'].isin(store_data['merchant_id'])]

    # 显示合并前的数据大小和前三行
    print(f"{test_file} 合并前的数据大小（行，列）:", test_data.shape)
    print("合并前数据的前三行:")
    print(test_data.head(3))

    # 使用左合并
    merged_data = pd.merge(test_data, store_data[['merchant_id', 'store_count']], on='merchant_id', how='left')
    
    # 转换store_count列的数据类型为整数，处理可能的NaN值
    merged_data['store_count'] = merged_data['store_count'].fillna(0).astype(int)

    # 显示合并后的数据大小和前三行
    print(f"{test_file} 合并后的数据大小（行，列）:", merged_data.shape)
    print("合并后数据的前三行:")
    print(merged_data.head(3))
    
    # 保存更新后的数据
    merged_data.to_csv(f"{test_file.replace('info.txt', 'info_with_store_count.txt')}", index=False)


测试1/test_both_with_merchant_info.txt 合并前的数据大小（行，列）: (65, 7)
合并前数据的前三行:
   apply_id      merchant_id           apply_date  lend_amount  lend_period  \
0  51179153  741067405523374  2018-05-23 00:00:00      50000.0            4   
1  51727931   72468430993589  2018-05-28 00:00:00       5000.0            6   
2  49518154  120072423392034  2018-05-02 00:00:00      50000.0           12   

   industry_level1  province  
0                2         1  
1                2         1  
2                1         1  
测试1/test_both_with_merchant_info.txt 合并后的数据大小（行，列）: (65, 8)
合并后数据的前三行:
   apply_id      merchant_id           apply_date  lend_amount  lend_period  \
0  51179153  741067405523374  2018-05-23 00:00:00      50000.0            4   
1  51727931   72468430993589  2018-05-28 00:00:00       5000.0            6   
2  49518154  120072423392034  2018-05-02 00:00:00      50000.0           12   

   industry_level1  province  store_count  
0                2         1            1  
1           

#####  训练数据

In [22]:
import pandas as pd

# 加载商户-店铺数据并确保merchant_id唯一
store_data = pd.read_csv('商户-店铺/merchant_count.txt')
store_data = store_data.drop_duplicates(subset=['merchant_id'], keep='first')

# 训练文件路径
train_files = [
    '训练1/train_both_with_merchant_info.txt',
    '训练1/train_combined_with_merchant_info.txt'
]

# 处理每个训练文件
for train_file in train_files:
    # 读取训练数据
    train_data = pd.read_csv(train_file)
    
    # 确保训练数据中的merchant_id在商户-店铺数据中能找到
    train_data = train_data[train_data['merchant_id'].isin(store_data['merchant_id'])]

    # 显示合并前的数据大小和前三行
    print(f"{train_file} 合并前的数据大小（行，列）:", train_data.shape)
    print("合并前数据的前三行:")
    print(train_data.head(3))

    # 使用左合并
    merged_data = pd.merge(train_data, store_data[['merchant_id', 'store_count']], on='merchant_id', how='left')
    
    # 转换store_count列的数据类型为整数，处理可能的NaN值
    merged_data['store_count'] = merged_data['store_count'].fillna(0).astype(int)

    # 显示合并后的数据大小和前三行
    print(f"{train_file} 合并后的数据大小（行，列）:", merged_data.shape)
    print("合并后数据的前三行:")
    print(merged_data.head(3))
    
    # 保存更新后的数据
    new_file_name = train_file.replace('info.txt', 'info_with_store_count.txt')
    merged_data.to_csv(new_file_name, index=False)

    print(f"更新后的数据已保存到 {new_file_name}")


训练1/train_both_with_merchant_info.txt 合并前的数据大小（行，列）: (164, 8)
合并前数据的前三行:
   apply_id      merchant_id           apply_date  lend_amount  lend_period  \
0  52849963  934378057859389  2018-06-08 00:00:00      50000.0            3   
1  51968426  671059822241463  2018-06-01 00:00:00       5000.0           12   
2  50007193  581824784033132  2018-05-08 00:00:00      45000.0            4   

   is_30days_overdue  industry_level1  province  
0                  0                1         2  
1                  1                1         2  
2                  0                1         2  
训练1/train_both_with_merchant_info.txt 合并后的数据大小（行，列）: (164, 9)
合并后数据的前三行:
   apply_id      merchant_id           apply_date  lend_amount  lend_period  \
0  52849963  934378057859389  2018-06-08 00:00:00      50000.0            3   
1  51968426  671059822241463  2018-06-01 00:00:00       5000.0           12   
2  50007193  581824784033132  2018-05-08 00:00:00      45000.0            4   

   is_30days_overdue

### 4、统计每个商户关联的店铺被抛出的次数

#### 新特征 pt_count_all

In [10]:
import pandas as pd

# 文件路径
file_paths = [
    '抛出点数据/table_throwpoint_test.txt',
    '抛出点数据/table_throwpoint_training.txt']

# 读取数据并打印各个文件的数据量
dataframes = []
for file in file_paths:
    df = pd.read_csv(file)
    print(f"{file} 的数据量为: {df.shape[0]}行")
    dataframes.append(df)

# 合并数据
combined_df = pd.concat(dataframes, ignore_index=True)
print(f"合并后的总数据量为: {combined_df.shape[0]}行")

# 去重数据
unique_df = combined_df.drop_duplicates()
print(f"去重后的总数据量为: {unique_df.shape[0]}行")

# 保存合并且去重后的数据到新文件
unique_df.to_csv('抛出点数据/combined.txt', index=False)


抛出点数据/table_throwpoint_test.txt 的数据量为: 6815行
抛出点数据/table_throwpoint_training.txt 的数据量为: 6733行
合并后的总数据量为: 13548行
去重后的总数据量为: 13548行


#### 4.1 查看有店铺被抛出好几次的情况，决定用计数处理这个文件

In [24]:
import pandas as pd

# 读取数据文件
file_path = '抛出点数据/combined.txt'
data = pd.read_csv(file_path)

# 统计每个store_id的不同pt的数量
store_id_pt_counts = data.groupby('store_id')['pt'].nunique()

# 找出有多个不同pt的store_id
multiple_pts_store_ids = store_id_pt_counts[store_id_pt_counts > 1]

# 打印这些store_id的数量
print(f"存在 {len(multiple_pts_store_ids)} 个store_id对应多个不同的pt。")

# 打印出几个示例
print("以下是一些有多个pt对应的store_id及其计数：")
print(multiple_pts_store_ids.head())  # 打印前几个示例


存在 3512 个store_id对应多个不同的pt。
以下是一些有多个pt对应的store_id及其计数：
store_id
222875327743     4
408693962279     3
1254938114014    4
1280226055936    2
1551164535618    7
Name: pt, dtype: int64


In [25]:
import pandas as pd

# 读取数据文件
file_path = '抛出点数据/combined.txt'
data = pd.read_csv(file_path)

# 统计每个store_id的不同pt的数量
store_id_pt_counts = data.groupby('store_id')['pt'].nunique().reset_index(name='pt_count')

# 合并pt计数到原始数据中
data_with_pt_count = pd.merge(data, store_id_pt_counts, on='store_id', how='left')

# 保存到新文件
output_file_path = '抛出点数据/combined_with_pt_counts.txt'
data_with_pt_count.to_csv(output_file_path, index=False)

print(f"已添加pt_count字段，并将数据保存到新文件 '{output_file_path}'。")


已添加pt_count字段，并将数据保存到新文件 '抛出点数据/combined_with_pt_counts.txt'。


In [26]:
#对于pt_count不等于1的不同的store_id，修改它对应的pt_count为1，再保存到原文件

import pandas as pd

# 读取数据文件
data_path = '抛出点数据/combined_with_pt_counts.txt'
data = pd.read_csv(data_path)

# 筛选 pt_count 不等于1的行
data_filtered = data[data['pt_count'] != 1]

# 获取不等于1的不同store_id的数量
unique_store_ids = data_filtered['store_id'].nunique()

# 打印结果
print(f"pt_count 不等于1的不同的store_id有 {unique_store_ids} 个。")

# 修改 pt_count 不等于1的所有store_id的pt_count为1
data.loc[data['pt_count'] != 1, 'pt_count'] = 1

# 保存修改后的数据到原文件
data.to_csv(data_path, index=False)

# 再次加载数据并确认修改
data_reloaded = pd.read_csv(data_path)
print(f"修改后再检查pt_count不等于1的行数: {data_reloaded[data_reloaded['pt_count'] != 1].shape[0]}")


pt_count 不等于1的不同的store_id有 3512 个。
修改后再检查pt_count不等于1的行数: 0


#### 4.2 把store对应的merchent_id加上去

In [27]:
import pandas as pd

# 读取抛出点数据
data_path = '抛出点数据/combined_with_pt_counts.txt'
data_combined = pd.read_csv(data_path)

# 读取商户-店铺数据
merchant_path = '商户-店铺/merchant_store_count.txt'
merchant_data = pd.read_csv(merchant_path)

# 获取两个文件中的store_id集合
data_store_ids = set(data_combined['store_id'])
merchant_store_ids = set(merchant_data['store_id'])

# 找出在data_combined中但不在merchant_data中的store_id
unmatched_store_ids = data_store_ids - merchant_store_ids

# 打印未匹配的store_id数量
print(f"在抛出点数据中有 {len(unmatched_store_ids)} 个store_id无法在商户-店铺数据中找到。")


在抛出点数据中有 0 个store_id无法在商户-店铺数据中找到。


In [28]:
import pandas as pd

# 读取抛出点数据
data_path = '抛出点数据/combined_with_pt_counts.txt'
data_combined = pd.read_csv(data_path)

# 读取商户-店铺数据
merchant_path = '商户-店铺/merchant_store_count.txt'
merchant_data = pd.read_csv(merchant_path)

# 假设商户-店铺文件包含 store_id 和 merchant_id 字段
# 根据 store_id 合并数据，添加 merchant_id
data_with_merchant = pd.merge(data_combined, merchant_data[['store_id', 'merchant_id']], on='store_id', how='left')

# 保存到新文件
output_file_path = '抛出点数据/combined_with_merchant.txt'
data_with_merchant.to_csv(output_file_path, index=False)

print(f"已将merchant_id添加到数据中，并保存在 '{output_file_path}'。")


已将merchant_id添加到数据中，并保存在 '抛出点数据/combined_with_merchant.txt'。


#### 4.3 计算每个merchant_id下的所有store_id的pt_count总和，即该商户旗下的所有店铺一共被抛出多少次

In [29]:
import pandas as pd

# 读取数据文件
data_path = '抛出点数据/combined_with_merchant.txt'
data = pd.read_csv(data_path)

# 计算不同的merchant_id的数量
unique_merchant_ids = data['merchant_id'].nunique()

# 打印结果
print(f"文件中有 {unique_merchant_ids} 个不同的merchant_id。")


文件中有 4593 个不同的merchant_id。


In [30]:
import pandas as pd

# 读取数据文件
data_path = '抛出点数据/combined_with_merchant.txt'
data = pd.read_csv(data_path)

# 删除重复的store_id行，只保留一个
data_unique_store_id = data.drop_duplicates(subset='store_id')

# 计算每个merchant_id的pt_count总和
pt_count_all = data.groupby('merchant_id')['pt_count'].sum().reset_index()
pt_count_all.rename(columns={'pt_count': 'pt_count_all'}, inplace=True)

# 合并pt_count_all回到去重后的数据中
data_final = pd.merge(data_unique_store_id, pt_count_all, on='merchant_id', how='left')

# 保存到新文件
output_file_path = '抛出点数据/pt_final.txt'
data_final.to_csv(output_file_path, index=False)

print(f"已处理数据，并保存到 '{output_file_path}'。")


已处理数据，并保存到 '抛出点数据/pt_final.txt'。


In [31]:
import pandas as pd

# 读取抛出点数据文件
data_path = '抛出点数据/pt_final.txt'
data = pd.read_csv(data_path)

# 删除 store_id 和 pt 字段
data.drop(['store_id', 'pt','pt_count'], axis=1, inplace=True)

# 对 merchant_id 进行降重，只保留第一次出现的行
data_unique = data.drop_duplicates(subset='merchant_id')

# 保存到新文件
output_file_path = '抛出点数据/pt.txt'
data_unique.to_csv(output_file_path, index=False)
print(data_unique.shape[0])
print(f"处理后的数据已保存到 '{output_file_path}'。")


4593
处理后的数据已保存到 '抛出点数据/pt.txt'。


In [32]:
import pandas as pd

# 文件路径
test_files = [
    '测试1/test_both_with_merchant_info_with_store_count.txt',
    '测试1/test_combined_with_merchant_info_with_store_count.txt',
    '测试1/test_history_with_merchant_info.txt',
    '测试1/test_neither_with_merchant_info.txt'
]
merchant_count_file = '抛出点数据/pt.txt'

# 读取商户ID数据
merchant_count_df = pd.read_csv(merchant_count_file)
merchant_ids_set = set(merchant_count_df['merchant_id'])

# 分别处理每个测试文件
for test_file in test_files:
    test_df = pd.read_csv(test_file)
    # 计算不能在merchant_count.txt中找到的merchant_id数量
    missing_count = test_df[~test_df['merchant_id'].isin(merchant_ids_set)].shape[0]
    print(f"{test_file} 中不能找到的merchant_id数量为: {missing_count}")


测试1/test_both_with_merchant_info_with_store_count.txt 中不能找到的merchant_id数量为: 41
测试1/test_combined_with_merchant_info_with_store_count.txt 中不能找到的merchant_id数量为: 222
测试1/test_history_with_merchant_info.txt 中不能找到的merchant_id数量为: 9
测试1/test_neither_with_merchant_info.txt 中不能找到的merchant_id数量为: 27


In [33]:
import pandas as pd

# 文件路径
test_files = [
    '训练1/train_both_with_merchant_info_with_store_count.txt',
    '训练1/train_combined_with_merchant_info_with_store_count.txt',
    '训练1/train_history_with_merchant_info.txt',
    '训练1/train_neither_with_merchant_info.txt'
]
merchant_count_file = '抛出点数据/pt.txt'

# 读取商户ID数据
merchant_count_df = pd.read_csv(merchant_count_file)
merchant_ids_set = set(merchant_count_df['merchant_id'])

# 分别处理每个测试文件
for test_file in test_files:
    test_df = pd.read_csv(test_file)
    # 计算不能在merchant_count.txt中找到的merchant_id数量
    missing_count = test_df[~test_df['merchant_id'].isin(merchant_ids_set)].shape[0]
    print(f"{test_file} 中不能找到的merchant_id数量为: {missing_count}")


训练1/train_both_with_merchant_info_with_store_count.txt 中不能找到的merchant_id数量为: 102
训练1/train_combined_with_merchant_info_with_store_count.txt 中不能找到的merchant_id数量为: 531
训练1/train_history_with_merchant_info.txt 中不能找到的merchant_id数量为: 31
训练1/train_neither_with_merchant_info.txt 中不能找到的merchant_id数量为: 66


: 

#### 缺失的太多，pt_count_all特征放弃